In [1]:
import torch
from torchvision import datasets, transforms


torch.manual_seed(73)

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./MNIST', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=64, shuffle=True)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./MNIST', train=False,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=1, shuffle=True)

In [2]:
import torch.nn as nn
import torch.optim as optim


class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.fc1 = nn.Linear(28*28, 256)
        self.fc2 = nn.Linear(256, 64)
        self.fc3 = nn.Linear(64, 10)
        
    def forward(self, x):
        out = x.reshape(-1, 28*28)
        out = self.fc1(out)
        out = out * out
        out = self.fc2(out)
        out = out * out
        out = self.fc3(out)
        out = nn.functional.log_softmax(out, dim=1)
        return out

In [3]:
def train(model, device, train_loader, optimizer, epochs):
    model.train()
    for epoch in range(1, epochs + 1):
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = nn.functional.nll_loss(output, target)
            loss.backward()
            optimizer.step()
            if batch_idx % 100 == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(data), len(train_loader.dataset),
                    100. * batch_idx / len(train_loader), loss.item()))
    model.eval()
    return model

In [4]:
model = Model()
optimizer = optim.Adadelta(model.parameters(), lr=0.001)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = train(model, device, train_loader, optimizer, 10)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.339463
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.291147
Train Epoch: 1 [12800/60000 (21%)]	Loss: 2.257251
Train Epoch: 1 [19200/60000 (32%)]	Loss: 2.198240
Train Epoch: 1 [25600/60000 (43%)]	Loss: 2.165964
Train Epoch: 1 [32000/60000 (53%)]	Loss: 2.082282
Train Epoch: 1 [38400/60000 (64%)]	Loss: 2.033545
Train Epoch: 1 [44800/60000 (75%)]	Loss: 1.958289
Train Epoch: 1 [51200/60000 (85%)]	Loss: 1.758407
Train Epoch: 1 [57600/60000 (96%)]	Loss: 1.509839
Train Epoch: 2 [0/60000 (0%)]	Loss: 1.639862
Train Epoch: 2 [6400/60000 (11%)]	Loss: 1.461483
Train Epoch: 2 [12800/60000 (21%)]	Loss: 1.226405
Train Epoch: 2 [19200/60000 (32%)]	Loss: 1.068823
Train Epoch: 2 [25600/60000 (43%)]	Loss: 1.307152
Train Epoch: 2 [32000/60000 (53%)]	Loss: 1.182487
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.769091
Train Epoch: 2 [44800/60000 (75%)]	Loss: 1.015962
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.648827
Train Epoch: 2 [57600/60000 (96%)]	Loss: 0.785544
Train Epoch:

In [5]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += nn.functional.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [6]:
test(model, device, test_loader)


Test set: Average loss: 0.3157, Accuracy: 9197/10000 (92%)



In [10]:
import tenseal as ts

class HEModel:
    def __init__(self, ts_context, fc1, fc2, fc3):
        self.ts_context = ts_context
        self.fc1_weight = fc1.weight.t().tolist()
        self.fc1_bias = fc1.bias.tolist()
        self.fc2_weight = fc2.weight.t().tolist()
        self.fc2_bias = fc2.bias.tolist()
        self.fc3_weight = fc3.weight.t().tolist()
        self.fc3_bias = fc3.bias.tolist()
        
    def forward(self, x):
        batch_size = x.shape[0]
        out = []
        for i in range(batch_size):
            # reshape single image into vector and encrypt it
            vec = x[i].flatten().tolist()
            encrypted_vec = ts.ckks_vector(self.ts_context, vec)
            # first fc layer + square activation function
            encrypted_vec = encrypted_vec.mm(self.fc1_weight) + self.fc1_bias
            encrypted_vec *= encrypted_vec
            # second fc layer + square activation function
            encrypted_vec = encrypted_vec.mm(self.fc2_weight) + self.fc2_bias
            encrypted_vec *= encrypted_vec
            # third fc layer
            encrypted_vec = encrypted_vec.mm(self.fc3_weight) + self.fc3_bias
            # decrypt
            out.append(encrypted_vec.decrypt())
            
        out = torch.tensor(out)
        out = nn.functional.log_softmax(out, dim=1)
        return out
    
    def __call__(self, x):
        return self.forward(x)


In [11]:
context = ts.context(ts.SCHEME_TYPE.CKKS, 16384, coeff_mod_bit_sizes=[40, 25, 25, 25, 25, 25, 40])
context.global_scale = 2 ** 25
context.generate_galois_keys()
he_model = HEModel(context, model.fc1, model.fc2, model.fc3)

In [12]:
from time import time

t_start = time()

test_loss = 0
correct = 0
count = 30
with torch.no_grad():
    for data, target in test_loader:
        count -= 1
        data, target = data.to(device), target.to(device)
        output = he_model(data)
        test_loss += nn.functional.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
        pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
        correct += pred.eq(target.view_as(pred)).sum().item()
        if count == 0:
            break
        
count=30
test_loss /= count

print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, count,
    100. * correct / count))

t_end = time()

TypeError: object of type 'int' has no len()

In [14]:
test_loss /= 30

print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, 30,
    100. * correct / 30))

t_end = time()


Test set: Average loss: 0.2521, Accuracy: 27/30 (90%)



In [19]:
(t_end - t_start) / 60

108.09027614196141

## Possible improvement

- Decrease hidden layers (256->128 and 64->32)
- Use smaller poly_modulus => use coeff mod [60, 40, 40, 60] with a scale of 20 so that we do 2 multiplication before rescaling
- restructure the diagonals is a single plaintext